In [3]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
from PreproccessPipeline import get_quantile_ranges, flattenArray


In [4]:
data = pd.read_csv('data/bike-sharing-hourly.csv')
data['dteday'] = pd.to_datetime(data['dteday'], format='%d/%m/%Y')
# data['dteday'] = data['dteday'].dt.strftime('%d').astype(int)
print(data['dteday'])



# these give you 100% accuracy, because they make up the cnt
data = data.drop(['casual', 'instant', 'registered'], axis=1)

data.dtypes
# data.head(50)
# data['class'] = pd.qcut(data['cnt'], q=4, labels=False)
# classCodes = dict(enumerate(data['class'].cat.categories))


continuous_features = ['dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']


quantileRanges = get_quantile_ranges(data['cnt'],4)
print(quantileRanges)


0        01/01/2011
1        01/01/2011
2        01/01/2011
3        01/01/2011
4        01/01/2011
            ...    
17374    31/12/2012
17375    31/12/2012
17376    31/12/2012
17377    31/12/2012
17378    31/12/2012
Name: dteday, Length: 17379, dtype: object
{1: [1.0, 40.0], 2: [40.0, 142.0], 3: [142.0, 281.0], 4: [281.0, 977.0]}


In [85]:
import pickle

filename = "models/bike-share-nn.pickle"
clf = pickle.load(open(filename, "rb"))


In [86]:
import dice_ml
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

print(continuous_features)
# Dataset for training an ML model
d = dice_ml.Data(dataframe=data,
                 continuous_features=continuous_features,
                 outcome_name='cnt')

# Pre-trained ML model
m = dice_ml.Model(model=clf, backend="sklearn", model_type='regressor')
# DiCE explanation instance
exp = dice_ml.Dice(d, m, method="genetic",)


['dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']


In [87]:
import json
import csv


percentage_to_sample = 1 / (len(quantileRanges) - 1)
# percentage_to_sample = percentage_to_sample * 0.1
f = open('counterfactuals/bike-share-1.csv', 'w')
first = True
writer = csv.writer(f)
for q in quantileRanges:
    print(quantileRanges[q])
    query_instances = data[(data['cnt'] >= quantileRanges[q][0]) & (
        data['cnt'] <= quantileRanges[q][1])]

    for desiredRange in quantileRanges:
        if (desiredRange == q):
            continue
        print(
            f" {q_range} => {desiredRange}({percentage_to_sample})")

        result = exp.generate_counterfactuals(
            query_instances.drop(['cnt'], axis=1).sample(
                frac=percentage_to_sample, random_state=1),
            total_CFs=1,
            desired_range=quantileRanges[desiredRange],
            # proximity_weight=0.8,
            # sparsity_weight=0.2,
            # diversity_weight=0.6,
            # categorical_penalty=0.1,
            # stopping_threshold=0.4
        )
        result = json.loads(result.to_json())
        cfList = result["cfs_list"]
        cfList = flattenArray(cfList)
        
        if (first):
            print(result['feature_names_including_target'])
            writer.writerow(result['feature_names_including_target'])
            first = False
        writer.writerows(cfList)


[1.0, 40.0]
 4 => 2(0.3333333333333333)


 20%|██        | 294/1459 [00:33<02:11,  8.87it/s]


KeyboardInterrupt: 